In [22]:
import re
from pathlib import Path

import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report


In [23]:
# ↳ edit these to match your locations
MANUAL_XLSX = Path("/Users/srinivasana/Downloads/peds_filtered_10_each_A_JB.xlsx")
LLM_CSV     = Path("/Users/srinivasana/Documents/peds_agents/llm_predictions_o3-mini_v2_test.csv")


In [24]:
# Grab every 5- to 7-digit NDA / BLA / ANDA number in a cell
ID_PAT = re.compile(r"\d{5,7}")

def extract_ids(cell):
    if pd.isna(cell):
        return []
    return ID_PAT.findall(str(cell))

def normalise(lbl):
    """map various spellings to canonical labels"""
    if not isinstance(lbl, str):
        return lbl
    lbl = lbl.strip().lower()
    mapping = {"none": "None", "partial": "Partial", "full": "Full",
               "unlabelled": "Unlabeled", "unlabeled": "Unlabeled"}
    return mapping.get(lbl, lbl.capitalize())


In [5]:
llm = pd.read_csv(LLM_CSV, dtype=str)

# make / clean app_id
if "app_id" not in llm.columns:
    # use the txt_file field if that’s where the ID is
    llm["app_id"] = llm["txt_file"].map(lambda s: extract_ids(s)[0] if s else None)

llm = llm.rename(columns={"resolved_label": "label_llm"})   # adjust if your column differs
llm = llm[["app_id", "label_llm"]].dropna(subset=["app_id"])
llm["app_id"] = llm["app_id"].astype(str)
llm["label_llm"] = llm["label_llm"].apply(normalise)

llm.head()


,app_id,label_llm
0,021505,NaN
1,022253,Partial
2,022518,NaN
3,050441,Partial
4,050684,Partial


In [6]:
llm

,app_id,label_llm
0,021505,NaN
1,022253,Partial
2,022518,NaN
3,050441,Partial
4,050684,Partial
5,125526,Partial
6,204275,NaN
7,205625,NaN
8,205641,NaN
9,205836,Partial


In [9]:
app_col_llm

'app_id'

In [9]:
# %% ───────────────────────────────────────────────────────────────────────────
# Notebook ▶  Manual vs LLM label agreement
# Author:  (your name)   Date: (today)
# %%

import re, pandas as pd

from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, f1_score, precision_score, recall_score
)

MANUAL_PATH = "/Users/srinivasana/Documents/peds_agents/peds_filtered_10_each_A_JB.xlsx"
LLM_PATH    = "/Users/srinivasana/Documents/peds_agents/llm_predictions_o3-mini_v5_test.csv"

# %% ───────────────────────────────────────────────────────────────────────────
# 1  Load the two tables
# %%

manual = pd.read_excel(
    MANUAL_PATH,
    engine="openpyxl",
    keep_default_na=False,   # ← do NOT treat the default NA tokens as NaN
    na_values=[""]           # ← empty string is still NaN; "None" is now kept
)

llm = pd.read_csv(
    LLM_PATH,
    keep_default_na=False,   # same here
    na_values=[""]
)

print("Manual rows:", len(manual))
print("LLM rows   :", len(llm))




# %% ───────────────────────────────────────────────────────────────────────────
# 2  Locate the “application number” column in each dataframe
# %%

def find_app_col(df):
    """
    Return the column name that looks like ‘FDA Application Number(s)’ etc.
    Raises KeyError if not found.
    """
    candidates = []
    for c in df.columns:
        name = c.lower().replace("\xa0", " ")  # normalize nbsp
        if "application" in name and "number" in name:
            candidates.append(c)
    # common short names
    for c in df.columns:
        if c.lower() in {"app_id", "applicationnumber", "nda_bla"}:
            candidates.append(c)
    if not candidates:
        raise KeyError("Could not find application-number column")
    return candidates[0]

app_col_manual = find_app_col(manual)
app_col_llm    = find_app_col(llm)

# %% ───────────────────────────────────────────────────────────────────────────
# 3  Extract first 5-/6-/7-digit NDA/BLA number per row ➜ app_id
# %%

# def extract_first_id(s: str) -> str | None:
#     if pd.isna(s):
#         return None
#     m = re.search(r"\d{5,7}", str(s))
#     return m.group(0) if m else None

# for df, col in [(manual, app_col_manual), (llm, app_col_llm)]:
#     df["app_id"] = df[col].apply(extract_first_id)

# # quick sanity
# print("\nSample IDs\n", manual["app_id"].head())
# ── 3. Extract & canonicalise the NDA/BLA number ────────────────────────────
def extract_app_id(cell) -> str | None:
    """
    Grab the first 5- to 7-digit sequence in `cell`.
    Return it as a canonical string with leading zeros stripped.
    """
    if pd.isna(cell):
        return None
    m = re.search(r"\d{5,7}", str(cell))
    if not m:
        return None
    return str(int(m.group()))          # int(...) drops any leading zeros

for df, col in [(manual, app_col_manual), (llm, app_col_llm)]:
    df["app_id"] = df[col].apply(extract_app_id)

# sanity-check
print("\nManual IDs  :", sorted(manual['app_id'].dropna().unique())[:8])
print("LLM IDs     :", sorted(llm['app_id'].dropna().unique())[:8])


# %% ───────────────────────────────────────────────────────────────────────────
# 4  Normalise label text
# %%

LABEL_COL_MANUAL = "resolved_label_A"               # ← adjust if you use another
LABEL_COL_LLM    = "resolved_label"               # column in llm csv

norm = {
    "none":      "None",
    "partial":   "Partial",
    "full":      "Full",
    "unlabeled": "Unlabeled",
}

for df, col in [(manual, LABEL_COL_MANUAL), (llm, LABEL_COL_LLM)]:
    df[col] = (df[col].astype(str)
                      .str.strip()
                      .str.lower()
                      .map(norm)
                      .fillna(df[col]))          # leave original if unmapped

# Drop rows without a (manual OR llm) label
manual = manual.dropna(subset=[LABEL_COL_MANUAL])
llm    = llm.dropna(subset=[LABEL_COL_LLM])

# %% ───────────────────────────────────────────────────────────────────────────
# 5  Inner-join on app_id and compare
# %%

df = manual.merge(
        llm[[ "app_id", LABEL_COL_LLM ]],
        on="app_id",
        how="inner",
        suffixes=("_manual", "_llm"),
)

print(f"\nJoined rows: {len(df)}  (manual∩llm)")

# %% ───────────────────────────────────────────────────────────────────────────
# 6  Confusion-matrix & metrics
# %%

labels = ["None", "Partial", "Full", "Unlabeled"]

cm = confusion_matrix(df[LABEL_COL_MANUAL], df[LABEL_COL_LLM], labels=labels)
cm_df = pd.DataFrame(
    cm,
    index=[f"man_{l}" for l in labels],
    columns=[f"llm_{l}" for l in labels]
)

print("\nConfusion matrix")
display(cm_df.style.background_gradient(cmap="Blues"))

print("\nClassification report\n")
print(
    classification_report(
        df[LABEL_COL_MANUAL],
        df[LABEL_COL_LLM],
        labels=labels,
        zero_division=0,
    )
)


y_true = df[LABEL_COL_MANUAL]
y_pred = df[LABEL_COL_LLM]

print("\nOverall metrics")
print(f" Accuracy          : {accuracy_score(y_true, y_pred):.3f}")
print(f" Macro-F1          : {f1_score(y_true, y_pred, average='macro'):.3f}")
print(f" Weighted-F1       : {f1_score(y_true, y_pred, average='weighted'):.3f}")
print(f" Macro-Precision   : {precision_score(y_true, y_pred, average='macro', zero_division=0):.3f}")
print(f" Macro-Recall      : {recall_score(y_true, y_pred, average='macro'):.3f}")


# %% ───────────────────────────────────────────────────────────────────────────
# 7  Show every mismatch for manual spot-check
# %%

mismatch = df[df[LABEL_COL_MANUAL] != df[LABEL_COL_LLM]]
print(f"\nMismatches: {len(mismatch)}")
display(mismatch[
    ["app_id", LABEL_COL_MANUAL, LABEL_COL_LLM]
].reset_index(drop=True))

# %% ───────────────────────────────────────────────────────────────────────────
# (Optional) Save mismatches for review
# %%



OUT = "/Users/srinivasana/Documents/peds_agents/mismatch_review.csv"
mismatch.to_csv(OUT, index=False)
print(f"\nMismatch rows written to {OUT}")


Manual rows: 24
LLM rows   : 23

Manual IDs  : ['204275', '205625', '205641', '205836', '206829', '208083', '208798', '208799']
LLM IDs     : ['125526', '204275', '205625', '205641', '205836', '206829', '208083', '208798']

Joined rows: 21  (manual∩llm)

Confusion matrix


,llm_None,llm_Partial,llm_Full,llm_Unlabeled
man_None,9,3,0,0
man_Partial,0,6,0,0
man_Full,0,0,3,0
man_Unlabeled,0,0,0,0



Classification report

              precision    recall  f1-score   support

        None       1.00      0.75      0.86        12
     Partial       0.67      1.00      0.80         6
        Full       1.00      1.00      1.00         3
   Unlabeled       0.00      0.00      0.00         0

    accuracy                           0.86        21
   macro avg       0.67      0.69      0.66        21
weighted avg       0.90      0.86      0.86        21


Overall metrics
 Accuracy          : 0.857
 Macro-F1          : 0.886
 Weighted-F1       : 0.861
 Macro-Precision   : 0.889
 Macro-Recall      : 0.917

Mismatches: 3


,app_id,resolved_label_A,resolved_label
0,209949,None,Partial
1,206829,None,Partial
2,214679,None,Partial



Mismatch rows written to /Users/srinivasana/Documents/peds_agents/mismatch_review.csv


In [31]:
manual['app_id']

0     209949
1     021505
2     761046
3     204275
4       None
5     022253
6     205625
7     208083
8     761055
9     206829
10    212102
11    209363
12    215650
13    050441
14    214679
15    761055
16    205836
17    050684
18    208798
19    208799
20    761122
21    022518
Name: app_id, dtype: object

In [32]:
llm['app_id']

0      21505
1      22253
2      22518
3      50441
4      50684
5     125526
6     204275
7     205625
8     205641
9     205836
10    206829
11    208083
12    208798
13    208799
14    209363
15    209949
16    212102
17    214679
18    215650
19    216185
20    761046
21    761055
22    761122
Name: app_id, dtype: object